# Voice Emotion Recognition Project - Modeling

#  I. FEATURES EXTRACTION

# 1. Importing Libraries

In [1]:
import librosa as lr
import librosa.display
import librosa 
import os
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt

import pickle

import seaborn as sns

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

import warnings

warnings.simplefilter("ignore")
pd.options.display.max_columns = None

# 2. Loading the audio files into dataframe

In [ ]:
# Create directory for audio files 
audio = "/Users/diebl/lab/FinalProject__RecognitionSpeechEmotion/audio_speech_actors_01-24/"
actor_folders = os.listdir(audio) #list files in audio directory
actor_folders.sort() 
actor_folders[0:5]

In [ ]:
# Extract emotions, actors and gender
emotion = []
gender = []
actor = []
file_path = []
for i in actor_folders:
    filename = os.listdir(audio + i) #iterate over Actor folders
    for f in filename: # go through files in Actor folder
        part = f.split('.')[0].split('-')
        emotion.append(int(part[2]))
        actor.append(int(part[6]))
        bg = int(part[6])
        if bg%2 == 0:
            bg = "female"
        else:
            bg = "male"
        gender.append(bg)
        file_path.append(audio + i + '/' + f)

In [ ]:
# Create the dataframe
audio_df = pd.DataFrame(emotion)
audio_df = audio_df.replace({1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'})
audio_df = pd.concat([pd.DataFrame(gender),audio_df,pd.DataFrame(actor)],axis=1)
audio_df.columns = ['gender','emotion','actor']
audio_df = pd.concat([audio_df,pd.DataFrame(file_path, columns = ['path'])],axis=1)
audio_df.head()

In [ ]:
# Ensure that columns are correct
pd.set_option('display.max_colwidth', -1)
#Emotion (01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised).
audio_df.sample(10)

In [ ]:
# Look at distribution of classes
audio_df.emotion.value_counts().plot(kind='bar')

In [ ]:
# Save the dataframe into csv
audio_df.to_csv('audio_df.csv',index = False)

In [2]:
audio_df = pd.read_csv('audio_df.csv')

# 3. Extract features : Mel Spectogramm with amplitude converted in Decibels

In [ ]:
 # Iterate over all audio files and extract Log Mel Spectogram mean values 
df = pd.DataFrame(columns=['mel_spectrogram'])

counter=0

for index,path in enumerate(audio_df.path):
    X, sample_rate = librosa.load(path, res_type='kaiser_fast',duration=3,sr=44100,offset=0.5)
    
    #get the mel-scaled spectrogram (transform both the y-axis (frequency) to log scale, 
    spectrogram = librosa.feature.melspectrogram(y=X, sr=sample_rate, n_mels=128,fmax=8000) 
    #and the “color” axis (amplitude) to Decibels, which is kinda the log scale of amplitudes.)
    db_spec = librosa.power_to_db(spectrogram)
    #temporally average spectrogram
    log_spectrogram = np.mean(db_spec, axis = 0)
        
    df.loc[counter] = [log_spectrogram]
    counter=counter+1   


In [ ]:
# Turn array into a list and concat with audio_df to get corresponding emotions labels
mel_24_8 = pd.concat([audio_df,pd.DataFrame(df['mel_spectrogram'].values.tolist())],axis=1)
mel_24_8 = mel_24_8.fillna(0)

In [ ]:
mel_24_8.info()

In [ ]:
mel_24_8.head()

In [ ]:
# Drop path column - no need for modeling
mel_24_8.drop(columns='path',inplace=True)

In [ ]:
# Save the dataframe into csv
mel_24_8.to_csv('mel_24_8.csv',index = False)

# II. Modeling

In [3]:
mel_24_8 = pd.read_csv('mel_24_8.csv')
mel_24_8.head()

,gender,emotion,actor,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258
0,male,neutral,1,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.367455,-76.372063,-76.384773,-76.384773,-76.173912,-75.719101,-75.941528,-75.642616,-75.908943,-75.830963,-75.750702,-74.779770,-72.831291,-71.263046,-66.522675,-66.223503,-70.020477,-72.228935,-72.313690,-73.103096,-73.700745,-74.247360,-72.630081,-70.088722,-71.006859,-71.960075,-64.492950,-45.909077,-40.032604,-41.116264,-43.536011,-40.618973,-35.053967,-32.517483,-32.092548,-32.517563,-33.537037,-33.267624,-35.378643,-40.179279,-42.373196,-43.332584,-42.339584,-40.890388,-37.123341,-35.466946,-33.285469,-32.946716,-32.992203,-33.490017,-35.945240,-41.896309,-47.815334,-50.323181,-45.502876,-38.289410,-38.666992,-41.103245,-41.816746,-42.414581,-41.487278,-39.022465,-35.519356,-33.715797,-33.540745,-34.016991,-34.092331,-32.436264,-31.850286,-33.650314,-38.193562,-42.978088,-45.402485,-45.206005,-44.586697,-45.936981,-45.795696,-40.709064,-37.872944,-38.022907,-38.164814,-39.510433,-41.684795,-42.941948,-44.585972,-46.579136,-47.636093,-48.323044,-49.249058,-43.662033,-37.236511,-35.375561,-34.821701,-34.251915,-34.473354,-34.055584,-33.987965,-34.936680,-39.287586,-45.576580,-48.068054,-49.168839,-48.626278,-44.114082,-38.876415,-37.762611,-39.959415,-45.457245,-51.322018,-53.238468,-53.367477,-43.064678,-39.122593,-39.563782,-40.458473,-39.314373,-39.660778,-40.883408,-41.750561,-42.409489,-42.489120,-42.409801,-43.510303,-43.150318,-42.641510,-42.718555,-43.860710,-45.219101,-45.628616,-46.638668,-47.701420,-47.830730,-49.362091,-51.617405,-53.486431,-56.296726,-57.342640,-57.266979,-59.023403,-59.809151,-60.092953,-60.296028,-61.951149,-63.786072,-66.110413,-68.031937,-69.601761,-69.552559,-70.200996,-72.295227,-73.251480,-73.473793,-73.186089,-73.507355,-73.793976,-74.203773,-72.618195,-72.961769,-74.726311,-75.683647,-75.076500,-74.199730,-74.634224,-75.398056,-75.457855,-74.936638,-74.332146,-74.989464,-75.444618,-75.905334,-75.710159,-76.182739,-76.384773,-76.374603,-75.929733,-75.408356,-75.852905,-76.266586,-76.200012,-76.140991,-75.617950,-75.555855,-75.830528,-75.977928,-75.699814,-75.595741,-75.769661,-75.343491,-75.185760,-75.273163,-75.540672,-76.038315,-76.283257,-76.320847,-76.215599,-76.230431,-76.285728,-76.104240,-76.190956,-75.838509,-75.736618,-75.965042,-76.136429,-76.307678,-76.327888,-76.380356,-76.322456,-76.256866,-76.058685,-76.228973,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,male,neutral,1,-75.335518,-75.445320,-75.554031,-75.203949,-75.230530,-75.319374,-75.653793,-75.720039,-75.720039,-75.720039,-75.720039,-75.720039,-75.556015,-75.118057,-75.059258,-75.370628,-75.391228,-75.651840,-75.7

## 1. Let's run our model without preprocessing our datas

In [4]:
# drop the target value
X = mel_24_8.drop('emotion', axis=1)
y = mel_24_8['emotion']


In [5]:
#encoding gender feature
X= pd.get_dummies(X)
X.head()

,actor,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,gender_female,gender_male
0,1,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.367455,-76.372063,-76.384773,-76.384773,-76.173912,-75.719101,-75.941528,-75.642616,-75.908943,-75.830963,-75.750702,-74.779770,-72.831291,-71.263046,-66.522675,-66.223503,-70.020477,-72.228935,-72.313690,-73.103096,-73.700745,-74.247360,-72.630081,-70.088722,-71.006859,-71.960075,-64.492950,-45.909077,-40.032604,-41.116264,-43.536011,-40.618973,-35.053967,-32.517483,-32.092548,-32.517563,-33.537037,-33.267624,-35.378643,-40.179279,-42.373196,-43.332584,-42.339584,-40.890388,-37.123341,-35.466946,-33.285469,-32.946716,-32.992203,-33.490017,-35.945240,-41.896309,-47.815334,-50.323181,-45.502876,-38.289410,-38.666992,-41.103245,-41.816746,-42.414581,-41.487278,-39.022465,-35.519356,-33.715797,-33.540745,-34.016991,-34.092331,-32.436264,-31.850286,-33.650314,-38.193562,-42.978088,-45.402485,-45.206005,-44.586697,-45.936981,-45.795696,-40.709064,-37.872944,-38.022907,-38.164814,-39.510433,-41.684795,-42.941948,-44.585972,-46.579136,-47.636093,-48.323044,-49.249058,-43.662033,-37.236511,-35.375561,-34.821701,-34.251915,-34.473354,-34.055584,-33.987965,-34.936680,-39.287586,-45.576580,-48.068054,-49.168839,-48.626278,-44.114082,-38.876415,-37.762611,-39.959415,-45.457245,-51.322018,-53.238468,-53.367477,-43.064678,-39.122593,-39.563782,-40.458473,-39.314373,-39.660778,-40.883408,-41.750561,-42.409489,-42.489120,-42.409801,-43.510303,-43.150318,-42.641510,-42.718555,-43.860710,-45.219101,-45.628616,-46.638668,-47.701420,-47.830730,-49.362091,-51.617405,-53.486431,-56.296726,-57.342640,-57.266979,-59.023403,-59.809151,-60.092953,-60.296028,-61.951149,-63.786072,-66.110413,-68.031937,-69.601761,-69.552559,-70.200996,-72.295227,-73.251480,-73.473793,-73.186089,-73.507355,-73.793976,-74.203773,-72.618195,-72.961769,-74.726311,-75.683647,-75.076500,-74.199730,-74.634224,-75.398056,-75.457855,-74.936638,-74.332146,-74.989464,-75.444618,-75.905334,-75.710159,-76.182739,-76.384773,-76.374603,-75.929733,-75.408356,-75.852905,-76.266586,-76.200012,-76.140991,-75.617950,-75.555855,-75.830528,-75.977928,-75.699814,-75.595741,-75.769661,-75.343491,-75.185760,-75.273163,-75.540672,-76.038315,-76.283257,-76.320847,-76.215599,-76.230431,-76.285728,-76.104240,-76.190956,-75.838509,-75.736618,-75.965042,-76.136429,-76.307678,-76.327888,-76.380356,-76.322456,-76.256866,-76.058685,-76.228973,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1
1,1,-75.335518,-75.445320,-75.554031,-75.203949,-75.230530,-75.319374,-75.653793,-75.720039,-75.720039,-75.720039,-75.720039,-75.720039,-75.556015,-75.118057,-75.059258,-75.370628,-75.391228,-75.651840,-75.720039,-75.7

In [ ]:
X.info()

In [ ]:
X.describe()

In [6]:
X1 = X.copy()

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# Defining a function that will apply the model to the train and test sets and proving the results

def MLPClassifierModeling(X_train, X_test, y_train, y_test):
    #modeling
    model = MLPClassifier(hidden_layer_sizes=(200,), learning_rate='adaptive', max_iter=400)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    #accuracy measures
    accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
    print("Accuracy of the Recognizer is: {:.1f}%".format(accuracy*100))
    emotions = ['neutral','calm', 'happy', 'sad', 'angry', 'fear', 'disgust', 'surprise']
    print(classification_report(y_test, y_pred, labels=['neutral','calm', 'happy', 'sad', 'angry', 'fear', 'disgust', 'surprise']))
    
    #emotions detected
    emotions_reelles =y_test[10:20]
    emotions_predites = y_pred[10:20]
    df_er = pd.DataFrame(emotions_reelles).rename(columns ={'emotion' :'actual_emotion'}).reset_index(drop=True)
    df_ep =pd.DataFrame(emotions_predites, columns =['predicted_emotion'])
    emotions_concat= pd.concat([df_er.reset_index(drop=True), df_ep.reset_index(drop=True)], axis=1)
    return(emotions_concat)
    
    

In [9]:
MLPClassifierModeling(X_train, X_test, y_train, y_test)

Accuracy of the Recognizer is: 23.3%
              precision    recall  f1-score   support

     neutral       0.00      0.00      0.00        20
        calm       0.44      0.18      0.26        44
       happy       0.12      0.26      0.16        34
         sad       0.27      0.10      0.15        39
       angry       0.79      0.36      0.49        42
        fear       0.15      0.25      0.18        32
     disgust       0.19      0.41      0.26        32
    surprise       0.27      0.22      0.24        45

    accuracy                           0.23       288
   macro avg       0.28      0.22      0.22       288
weighted avg       0.31      0.23      0.24       288



,actual_emotion,predicted_emotion
0,disgust,disgust
1,surprise,surprise
2,sad,disgust
3,fear,angry
4,sad,disgust
5,disgust,disgust
6,angry,fear
7,fear,fear
8,calm,sad
9,disgust,disgust


Our model gives imbalanced results : good at predicting two emotions, not so good for the others.

## 2. Improving the model with standardization

In [10]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y, test_size=0.2, random_state=42)

In [11]:
#standardization

transformer = StandardScaler()
transformer.fit(X1_train)

# saving in a pickle
with open('std_transformer1.pickle', 'wb') as file:
    pickle.dump(transformer, file)
    
# loading from a pickle  
with open('std_transformer1.pickle', 'rb') as file:
    loaded_standardized = pickle.load(file)

X1_train = loaded_standardized.transform(X1_train)
X1_test = loaded_standardized.transform(X1_test)

In [12]:
MLPClassifierModeling(X1_train, X1_test, y1_train, y1_test)

Accuracy of the Recognizer is: 45.5%
              precision    recall  f1-score   support

     neutral       0.30      0.35      0.33        20
        calm       0.67      0.68      0.67        44
       happy       0.30      0.47      0.37        34
         sad       0.34      0.26      0.29        39
       angry       0.56      0.57      0.56        42
        fear       0.39      0.41      0.40        32
     disgust       0.45      0.47      0.46        32
    surprise       0.55      0.36      0.43        45

    accuracy                           0.45       288
   macro avg       0.45      0.45      0.44       288
weighted avg       0.47      0.45      0.45       288



,actual_emotion,predicted_emotion
0,disgust,disgust
1,surprise,surprise
2,sad,disgust
3,fear,angry
4,sad,sad
5,disgust,angry
6,angry,happy
7,fear,happy
8,calm,sad
9,disgust,disgust


Based on the classification report, our model has improved in detecting more emotions and also the predictions are more balanced. However, the sampled results show that some errors can be made : angry or fear predicted as happy.

## 3. With other features in addition to mel Spectogramm : Chroma and MFCCS

### a. Extracting the features into a dataframe

In [ ]:
# Iterate over all audio files and extract Log Mel Spectogram mean values 
df1 = pd.DataFrame(columns=['mel_spectro' ])

counter=0

for index,path in enumerate(audio_df.path):
    X, sample_rate = librosa.load(path, res_type='kaiser_fast',duration=3,sr=44100,offset=0.5)
    
    #get the mel-scaled spectrogram (transform both the y-axis (frequency) to log scale, 
    
    #spectrogram = librosa.feature.melspectrogram(y=X, sr=sample_rate, n_mels=128,fmax=8000)
    mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
  
        
    df1.loc[counter] = [mel]
    counter=counter+1   
    

In [ ]:
df1.info()

In [ ]:
# Iterate over all audio files and extract chroma mean values 
df2 = pd.DataFrame(columns=['chroma' ])
stft=np.abs(librosa.stft(X))
counter=0

for index,path in enumerate(audio_df.path):
    X, sample_rate = librosa.load(path, res_type='kaiser_fast',duration=3,sr=44100,offset=0.5)
    
    chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
  
        
    df2.loc[counter] = [chroma]
    counter=counter+1   
    

In [ ]:
df2.info()

In [ ]:
# Iterate over all audio files and extract MFCCS mean values 
 
df3 = pd.DataFrame(columns=['mfccs' ])

counter=0

for index,path in enumerate(audio_df.path):
    X, sample_rate = librosa.load(path, res_type='kaiser_fast',duration=3,sr=44100,offset=0.5)
    
    mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
  
        
    df3.loc[counter] = [mfccs]
    counter=counter+1   
    

In [ ]:
df3.info()

In [ ]:
# Turn array into a list and concat with audio_df to get corresponding emotions labels
df_features = pd.concat([audio_df,pd.DataFrame(df1['mel_spectro'].values.tolist()),pd.DataFrame(df2['chroma'].values.tolist()),pd.DataFrame(df3['mfccs'].values.tolist())],axis=1)
d_features= df_features.fillna(0)

In [ ]:
# Drop path column for modeling
df_features.drop(columns='path',inplace=True)

In [ ]:
df_features.to_csv('mel_chroma_mfccs_24_8.csv',index = False)

### b. First modeling without any preprocessing techniques

In [15]:
df_features = pd.read_csv('mel_chroma_mfccs_24_8.csv')
df_features.head()

,gender,emotion,actor,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,0.1,1.1,2.1,3.1,4.1,5.1,6.1,7.1,8.1,9.1,10.1,11.1,0.2,1.2,2.2,3.2,4.2,5.2,6.2,7.2,8.2,9.2,10.2,11.2,12.1,13.1,14.1,15.1,16.1,17.1,18.1,19.1,20.1,21.1,22.1,23.1,24.1,25.1,26.1,27.1,28.1,29.1,30.1,31.1,32.1,33.1,34.1,35.1,36.1,37.1,38.1,39.1
0,male,neutral,1,0.000011,0.001976,0.015901,0.032733,0.021538,0.008141,0.028234,0.022807,0.027077,0.012902,0.009569,0.012302,0.031973,0.054504,0.019130,0.015066,0.030320,0.030435,0.004375,0.005981,0.017045,0.012478,0.001975,0.000893,0.000955,0.000317,0.000230,0.000318,0.000686,0.001012,0.000662,0.000416,0.000470,0.001840,0.001414,0.002363,0.004953,0.003133,0.000804,0.000613,0.000905,0.000605,0.000758,0.001025,0.000851,0.001170,0.001033,0.000743,0.000513,0.000351,0.000561,0.001351,0.000298,0.000075,0.000148,0.000121,0.000150,0.000320,0.000749,0.000711,0.000604,0.000287,0.000130,0.000111,0.000106,0.000123,0.000204,0.000218,0.000306,0.000428,0.000435,0.000304,0.000081,0.000025,0.000016,0.000017,0.000041,0.000052,0.000051,0.000099,0.000075,0.000066,0.000044,0.000027,0.000033,0.000044,0.000028,0.000020,0.000013,0.000009,0.000006,0.000005,0.000005,0.000006,0.000008,0.000005,0.000003,0.000003,0.000003,0.000004,0.000003,0.000002,0.000004,0.000007,0.000007,0.000007,0.000004,0.000004,0.000004,0.000003,0.000002,0.000002,0.000002,0.000001,0.000001,2.208401e-06,2.137728e-06,1.075318e-06,9.700138e-07,6.827393e-07,4.325309e-07,2.405590e-07,1.063549e-07,2.829305e-08,5.515081e-09,1.295668e-09,1.544706e-10,1.205801e-11,0.767402,0.763228,0.72857,0.724856,0.720314,0.717113,0.749522,0.782215,0.7834,0.762521,0.756099,0.753244,-696.201050,78.316574,3.646741,13.573399,7.353319,14.752226,-4.387536,3.756527,-5.253314,-2.098398,-10.855770,0.928096,1.311460,-2.799618,2.291457,0.261636,-3.436544,2.309578,1.377719,-3.211596,-1.554741,-1.462649,0.523368,-1.446065,-4.471434,-1.687530,0.057159,-3.259277,-3.695817,-0.504393,-0.320704,-3.949646,-1.170431,-0.233570,-1.211523,0.016120,-1.144806,-2.061026,-1.967518,-1.471878
1,male,neutral,1,0.000014,0.001849,0.014971,0.047551,0.035372,0.006433,0.024542,0.038484,0.046205,0.012145,0.007730,0.015324,0.038466,0.056573,0.024035,0.014246,0.015317,0.029662,0.013318,0.003089,0.009146,0.021038,0.010059,0.000699,0.000492,0.000535,0.000354,0.000206,0.000432,0.000702,0.000895,0.000986,0.000334,0.000572,0.002868,0.006152,0.003517,0.002318,0.001857,0.001007,0.000570,0.001108,0.001184,0.000696,0.000820,0.001295,0.001075,0.000377,0.000651,0.000357,0.000218,0.000560,0.000125,0.000099,0.000156,0.000107,0.000106,0.000223,0.000280,0.000399,0.000580,0.000419,0.000137,0.000121,0.000078,0.000087,0.000139,0.000186,0.000277,0.000247,0.000399,0.000407,0.000226,0.000058,0.000020,0.000018,0.000045,0.000095,0.000058,0.000035,0.000051,0.000070,0.000035,0.000025,0.000027,0.000029,0.000040,0.000033,0.000019,0.000015,0.000006,0.000006,0.000004,0.000007,0.000008,0.000005,0.000005,0.000005,0.000006,0.000006,0.000006,0.000006,0.000010,0.000009,0.000010,0.000008,0.000009,0.000009,0.000006,0.000005,0.000005,0.000004,0.000003,0.000003,0.000002,3.780466e-06,2.841544e-06,1.463533e-06,9.996738e-07,5.547378e-07,3.049770e-07,2.454552e-07,1.095360e-07,3.616852e-08,8.127561e-09,1.619134e-09,2.312167e-10,1.308781e-11,0.767402,0.763228,0.72857,0.724856,0.720314,0.717113,0.749522,0.782215,0.7834,0.762521,0.756099,0.753244,-686.312988,78.732323,0.704220,14.193933,9.702352,14.567476,-5.940362,5.089293,-6.562455,-4.723670,-10.431581,1.062931,1.210753,-3.069765,1.290678,-0.721283,-3.504915,1.528015,0.505965,-3.727174,-1.899415,-1.406145,-0.521805,-0.254449,-3.945989,-2.609072,0.535067,-4.018624,-3.540811,0.265021,

In [16]:
# drop the target value
X = df_features.drop('emotion', axis=1)
y = df_features['emotion']

In [17]:
#encoding gender feature
X= pd.get_dummies(X)
X.head()

,actor,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,0.1,1.1,2.1,3.1,4.1,5.1,6.1,7.1,8.1,9.1,10.1,11.1,0.2,1.2,2.2,3.2,4.2,5.2,6.2,7.2,8.2,9.2,10.2,11.2,12.1,13.1,14.1,15.1,16.1,17.1,18.1,19.1,20.1,21.1,22.1,23.1,24.1,25.1,26.1,27.1,28.1,29.1,30.1,31.1,32.1,33.1,34.1,35.1,36.1,37.1,38.1,39.1,gender_female,gender_male
0,1,0.000011,0.001976,0.015901,0.032733,0.021538,0.008141,0.028234,0.022807,0.027077,0.012902,0.009569,0.012302,0.031973,0.054504,0.019130,0.015066,0.030320,0.030435,0.004375,0.005981,0.017045,0.012478,0.001975,0.000893,0.000955,0.000317,0.000230,0.000318,0.000686,0.001012,0.000662,0.000416,0.000470,0.001840,0.001414,0.002363,0.004953,0.003133,0.000804,0.000613,0.000905,0.000605,0.000758,0.001025,0.000851,0.001170,0.001033,0.000743,0.000513,0.000351,0.000561,0.001351,0.000298,0.000075,0.000148,0.000121,0.000150,0.000320,0.000749,0.000711,0.000604,0.000287,0.000130,0.000111,0.000106,0.000123,0.000204,0.000218,0.000306,0.000428,0.000435,0.000304,0.000081,0.000025,0.000016,0.000017,0.000041,0.000052,0.000051,0.000099,0.000075,0.000066,0.000044,0.000027,0.000033,0.000044,0.000028,0.000020,0.000013,0.000009,0.000006,0.000005,0.000005,0.000006,0.000008,0.000005,0.000003,0.000003,0.000003,0.000004,0.000003,0.000002,0.000004,0.000007,0.000007,0.000007,0.000004,0.000004,0.000004,0.000003,0.000002,0.000002,0.000002,0.000001,0.000001,2.208401e-06,2.137728e-06,1.075318e-06,9.700138e-07,6.827393e-07,4.325309e-07,2.405590e-07,1.063549e-07,2.829305e-08,5.515081e-09,1.295668e-09,1.544706e-10,1.205801e-11,0.767402,0.763228,0.72857,0.724856,0.720314,0.717113,0.749522,0.782215,0.7834,0.762521,0.756099,0.753244,-696.201050,78.316574,3.646741,13.573399,7.353319,14.752226,-4.387536,3.756527,-5.253314,-2.098398,-10.855770,0.928096,1.311460,-2.799618,2.291457,0.261636,-3.436544,2.309578,1.377719,-3.211596,-1.554741,-1.462649,0.523368,-1.446065,-4.471434,-1.687530,0.057159,-3.259277,-3.695817,-0.504393,-0.320704,-3.949646,-1.170431,-0.233570,-1.211523,0.016120,-1.144806,-2.061026,-1.967518,-1.471878,0,1
1,1,0.000014,0.001849,0.014971,0.047551,0.035372,0.006433,0.024542,0.038484,0.046205,0.012145,0.007730,0.015324,0.038466,0.056573,0.024035,0.014246,0.015317,0.029662,0.013318,0.003089,0.009146,0.021038,0.010059,0.000699,0.000492,0.000535,0.000354,0.000206,0.000432,0.000702,0.000895,0.000986,0.000334,0.000572,0.002868,0.006152,0.003517,0.002318,0.001857,0.001007,0.000570,0.001108,0.001184,0.000696,0.000820,0.001295,0.001075,0.000377,0.000651,0.000357,0.000218,0.000560,0.000125,0.000099,0.000156,0.000107,0.000106,0.000223,0.000280,0.000399,0.000580,0.000419,0.000137,0.000121,0.000078,0.000087,0.000139,0.000186,0.000277,0.000247,0.000399,0.000407,0.000226,0.000058,0.000020,0.000018,0.000045,0.000095,0.000058,0.000035,0.000051,0.000070,0.000035,0.000025,0.000027,0.000029,0.000040,0.000033,0.000019,0.000015,0.000006,0.000006,0.000004,0.000007,0.000008,0.000005,0.000005,0.000005,0.000006,0.000006,0.000006,0.000006,0.000010,0.000009,0.000010,0.000008,0.000009,0.000009,0.000006,0.000005,0.000005,0.000004,0.000003,0.000003,0.000002,3.780466e-06,2.841544e-06,1.463533e-06,9.996738e-07,5.547378e-07,3.049770e-07,2.454552e-07,1.095360e-07,3.616852e-08,8.127561e-09,1.619134e-09,2.312167e-10,1.308781e-11,0.767402,0.763228,0.72857,0.724856,0.720314,0.717113,0.749522,0.782215,0.7834,0.762521,0.756099,0.753244,-686.312988,78.732323,0.704220,14.193933,9.702352,14.567476,-5.940362,5.089293,-6.562455,-4.723670,-10.431581,1.062931,1.210753,-3.069765,1.290678,-0.721283,-3.504915,1.528015,0.505965,-3.727174,-1.899415,-1.406145,-0.521805,-0.254449,-3.945989,-2.609072,0.535067,-4.018624,-3.540811,0.265021,-1.516942,-

In [ ]:
X.describe()

In [18]:
X1 = X.copy()

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
MLPClassifierModeling(X_train, X_test, y_train, y_test)

Accuracy of the Recognizer is: 49.7%
              precision    recall  f1-score   support

     neutral       0.33      0.35      0.34        20
        calm       0.58      0.77      0.66        44
       happy       0.29      0.56      0.38        34
         sad       1.00      0.05      0.10        39
       angry       0.91      0.48      0.62        42
        fear       0.53      0.75      0.62        32
     disgust       0.65      0.34      0.45        32
    surprise       0.46      0.58      0.51        45

    accuracy                           0.50       288
   macro avg       0.59      0.49      0.46       288
weighted avg       0.62      0.50      0.47       288



,actual_emotion,predicted_emotion
0,disgust,happy
1,surprise,surprise
2,sad,calm
3,fear,fear
4,sad,calm
5,disgust,happy
6,angry,angry
7,fear,surprise
8,calm,calm
9,disgust,happy


### c. Improving the model with standardization

In [21]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y, test_size=0.2, random_state=42)

In [22]:
#standardization

transformer = StandardScaler()
transformer.fit(X1_train)

# saving in a pickle
with open('std_transformer2.pickle', 'wb') as file:
    pickle.dump(transformer, file)
    
# loading from a pickle  
with open('std_transformer2.pickle', 'rb') as file:
    loaded_standardized = pickle.load(file)

X1_train = loaded_standardized.transform(X1_train)
X1_test = loaded_standardized.transform(X1_test)

In [23]:
MLPClassifierModeling(X1_train, X1_test, y1_train, y1_test)

Accuracy of the Recognizer is: 73.3%
              precision    recall  f1-score   support

     neutral       0.68      0.85      0.76        20
        calm       0.82      0.84      0.83        44
       happy       0.49      0.62      0.55        34
         sad       0.81      0.77      0.79        39
       angry       0.81      0.83      0.82        42
        fear       0.74      0.72      0.73        32
     disgust       0.71      0.75      0.73        32
    surprise       0.80      0.53      0.64        45

    accuracy                           0.73       288
   macro avg       0.73      0.74      0.73       288
weighted avg       0.74      0.73      0.73       288



,actual_emotion,predicted_emotion
0,disgust,happy
1,surprise,surprise
2,sad,sad
3,fear,happy
4,sad,sad
5,disgust,disgust
6,angry,fear
7,fear,fear
8,calm,calm
9,disgust,disgust


Good quality of predictions with a good improvment of our accuracy score. However fear and disgust still predicted as happy. The model seems to detected strong emotions but is not able to determine if they are positive or negative emotions.

### d. Removing neutral that is generally not well predicted by our model

**Create the dataframe**

In [ ]:
df_features = pd.read_csv('mel_chroma_mfccs_24_8.csv')
df_features.head()

In [ ]:
filtered = df_features.loc[
                        (df_features["emotion"] =='calm')|
                        (df_features["emotion"] =='happy')|
                        (df_features["emotion"] =='sad')|
                        (df_features["emotion"] =='angry')|
                        (df_features["emotion"] =='fear')|
                        (df_features["emotion"] =='disgust')|
                        (df_features["emotion"] =='surprise')]
print(filtered)

In [ ]:
features_emotions = pd.DataFrame(filtered).reset_index(drop=True)
features_emotions.info()
features_emotions.head()

In [ ]:
# checking if neutral is not present in the dataframe
result = features_emotions.loc[
                        (features_emotions["emotion"] =='neutral')
                        ]
print(result)

In [ ]:
#Save the dataframe into csv
features_emotions.to_csv('mel_chroma_mfccs_24_7.csv',index = False)

**Modeling without preprocessing**

In [24]:
features_emotions= pd.read_csv('mel_chroma_mfccs_24_7.csv')
features_emotions.head()

,gender,emotion,actor,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,0.1,1.1,2.1,3.1,4.1,5.1,6.1,7.1,8.1,9.1,10.1,11.1,0.2,1.2,2.2,3.2,4.2,5.2,6.2,7.2,8.2,9.2,10.2,11.2,12.1,13.1,14.1,15.1,16.1,17.1,18.1,19.1,20.1,21.1,22.1,23.1,24.1,25.1,26.1,27.1,28.1,29.1,30.1,31.1,32.1,33.1,34.1,35.1,36.1,37.1,38.1,39.1
0,male,calm,1,0.000012,0.000789,0.008584,0.023643,0.024978,0.006251,0.016150,0.017182,0.019608,0.018035,0.006866,0.004610,0.010312,0.019593,0.020324,0.006712,0.001820,0.007984,0.009848,0.001885,0.000792,0.003688,0.004912,0.001009,0.000352,0.000334,0.000239,0.000103,0.000127,0.000227,0.000252,0.000423,0.000125,0.000209,0.000392,0.001427,0.001162,0.000189,0.000317,0.000813,0.000229,0.000284,0.000428,0.000135,0.000854,0.000672,0.000174,0.000263,0.000260,0.000224,0.000811,0.000487,0.000085,0.000091,0.000067,0.000084,0.000124,0.000143,0.000159,0.000330,0.000303,0.000175,0.000101,0.000054,0.000023,0.000015,0.000016,0.000022,0.000037,0.000052,0.000075,0.000098,0.000074,0.000029,0.000008,0.000007,0.000016,0.000021,0.000026,0.000028,0.000046,0.000040,0.000026,0.000037,0.000044,0.000061,0.000055,0.000045,0.000037,0.000014,0.000008,0.000007,0.000007,0.000007,0.000006,0.000008,0.000005,0.000003,0.000005,0.000004,0.000006,0.000008,0.000003,0.000004,0.000005,0.000004,0.000003,0.000004,0.000005,0.000003,0.000002,2.158778e-06,1.568582e-06,1.628014e-06,1.118427e-06,9.224078e-07,6.866002e-07,4.452201e-07,4.095236e-07,2.141846e-07,1.430198e-07,9.130326e-08,5.189249e-08,2.195618e-08,4.613697e-09,9.178104e-10,9.985994e-11,5.567851e-12,0.767402,0.763228,0.72857,0.724856,0.720314,0.717113,0.749522,0.782215,0.7834,0.762521,0.756099,0.753244,-727.783325,87.151093,5.981500,16.521162,7.651807,15.974822,-4.892543,5.531658,-8.175882,-0.301242,-11.974799,-0.529750,0.692740,-1.977111,-0.084285,0.115052,-3.832083,1.656441,2.074690,-4.249748,-1.789677,-0.458033,0.164515,-2.413014,-3.059638,-2.249351,-0.738288,-3.337880,-4.075123,-0.432539,-1.139909,-4.333735,-0.504630,-0.390943,-2.260401,0.173424,-1.041390,-1.922247,-0.495608,-1.814612
1,male,calm,1,0.000017,0.003102,0.016791,0.021407,0.029746,0.011495,0.012319,0.006782,0.013946,0.023442,0.009255,0.003194,0.007353,0.013187,0.030339,0.022584,0.006136,0.002648,0.003365,0.009951,0.005240,0.002188,0.002535,0.004221,0.002419,0.000305,0.000177,0.000130,0.000282,0.000397,0.000465,0.000744,0.000528,0.000417,0.000850,0.001355,0.000750,0.000438,0.000448,0.000182,0.000409,0.000865,0.000400,0.000325,0.000404,0.000284,0.000410,0.000225,0.000423,0.000388,0.000360,0.000634,0.000249,0.000061,0.000100,0.000058,0.000085,0.000101,0.000298,0.000244,0.000311,0.000146,0.000091,0.000040,0.000033,0.000017,0.000023,0.000031,0.000051,0.000069,0.000155,0.000118,0.000047,0.000015,0.000005,0.000003,0.000003,0.000010,0.000029,0.000029,0.000025,0.000030,0.000022,0.000020,0.000016,0.000030,0.000039,0.000029,0.000018,0.000013,0.000007,0.000005,0.000003,0.000004,0.000006,0.000005,0.000004,0.000003,0.000004,0.000004,0.000004,0.000004,0.000005,0.000007,0.000009,0.000008,0.000008,0.000006,0.000003,0.000002,0.000002,1.272558e-06,1.126081e-06,1.040018e-06,1.084160e-06,1.362575e-06,1.298073e-06,5.357613e-07,3.364116e-07,3.311025e-07,1.439164e-07,4.063457e-08,1.761115e-08,8.868716e-09,2.765349e-09,5.057543e-10,5.419542e-11,4.823885e-12,0.767402,0.763228,0.72857,0.724856,0.720314,0.717113,0.749522,0.782215,0.7834,0.762521,0.756099,0.753244,-704.303772,93.277084,9.311788,12.922705,7.454835,17.394232,-4.342582,3.875228,-8.110920,-0.968131,-9.430532,-1.396702,2.581494,-1.416424,0.146414,-0.556509,-3.518421,0.420847,2.156458,-3.789995,-2.339234,-1.558570,1.003638,-2.425308,-2.732662,-2.345526,-1.238754,

In [25]:
# drop the target value
X = features_emotions.drop('emotion', axis=1)
y = features_emotions['emotion']

In [26]:
#encoding gender feature
X= pd.get_dummies(X)
X.head()

,actor,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,0.1,1.1,2.1,3.1,4.1,5.1,6.1,7.1,8.1,9.1,10.1,11.1,0.2,1.2,2.2,3.2,4.2,5.2,6.2,7.2,8.2,9.2,10.2,11.2,12.1,13.1,14.1,15.1,16.1,17.1,18.1,19.1,20.1,21.1,22.1,23.1,24.1,25.1,26.1,27.1,28.1,29.1,30.1,31.1,32.1,33.1,34.1,35.1,36.1,37.1,38.1,39.1,gender_female,gender_male
0,1,0.000012,0.000789,0.008584,0.023643,0.024978,0.006251,0.016150,0.017182,0.019608,0.018035,0.006866,0.004610,0.010312,0.019593,0.020324,0.006712,0.001820,0.007984,0.009848,0.001885,0.000792,0.003688,0.004912,0.001009,0.000352,0.000334,0.000239,0.000103,0.000127,0.000227,0.000252,0.000423,0.000125,0.000209,0.000392,0.001427,0.001162,0.000189,0.000317,0.000813,0.000229,0.000284,0.000428,0.000135,0.000854,0.000672,0.000174,0.000263,0.000260,0.000224,0.000811,0.000487,0.000085,0.000091,0.000067,0.000084,0.000124,0.000143,0.000159,0.000330,0.000303,0.000175,0.000101,0.000054,0.000023,0.000015,0.000016,0.000022,0.000037,0.000052,0.000075,0.000098,0.000074,0.000029,0.000008,0.000007,0.000016,0.000021,0.000026,0.000028,0.000046,0.000040,0.000026,0.000037,0.000044,0.000061,0.000055,0.000045,0.000037,0.000014,0.000008,0.000007,0.000007,0.000007,0.000006,0.000008,0.000005,0.000003,0.000005,0.000004,0.000006,0.000008,0.000003,0.000004,0.000005,0.000004,0.000003,0.000004,0.000005,0.000003,0.000002,2.158778e-06,1.568582e-06,1.628014e-06,1.118427e-06,9.224078e-07,6.866002e-07,4.452201e-07,4.095236e-07,2.141846e-07,1.430198e-07,9.130326e-08,5.189249e-08,2.195618e-08,4.613697e-09,9.178104e-10,9.985994e-11,5.567851e-12,0.767402,0.763228,0.72857,0.724856,0.720314,0.717113,0.749522,0.782215,0.7834,0.762521,0.756099,0.753244,-727.783325,87.151093,5.981500,16.521162,7.651807,15.974822,-4.892543,5.531658,-8.175882,-0.301242,-11.974799,-0.529750,0.692740,-1.977111,-0.084285,0.115052,-3.832083,1.656441,2.074690,-4.249748,-1.789677,-0.458033,0.164515,-2.413014,-3.059638,-2.249351,-0.738288,-3.337880,-4.075123,-0.432539,-1.139909,-4.333735,-0.504630,-0.390943,-2.260401,0.173424,-1.041390,-1.922247,-0.495608,-1.814612,0,1
1,1,0.000017,0.003102,0.016791,0.021407,0.029746,0.011495,0.012319,0.006782,0.013946,0.023442,0.009255,0.003194,0.007353,0.013187,0.030339,0.022584,0.006136,0.002648,0.003365,0.009951,0.005240,0.002188,0.002535,0.004221,0.002419,0.000305,0.000177,0.000130,0.000282,0.000397,0.000465,0.000744,0.000528,0.000417,0.000850,0.001355,0.000750,0.000438,0.000448,0.000182,0.000409,0.000865,0.000400,0.000325,0.000404,0.000284,0.000410,0.000225,0.000423,0.000388,0.000360,0.000634,0.000249,0.000061,0.000100,0.000058,0.000085,0.000101,0.000298,0.000244,0.000311,0.000146,0.000091,0.000040,0.000033,0.000017,0.000023,0.000031,0.000051,0.000069,0.000155,0.000118,0.000047,0.000015,0.000005,0.000003,0.000003,0.000010,0.000029,0.000029,0.000025,0.000030,0.000022,0.000020,0.000016,0.000030,0.000039,0.000029,0.000018,0.000013,0.000007,0.000005,0.000003,0.000004,0.000006,0.000005,0.000004,0.000003,0.000004,0.000004,0.000004,0.000004,0.000005,0.000007,0.000009,0.000008,0.000008,0.000006,0.000003,0.000002,0.000002,1.272558e-06,1.126081e-06,1.040018e-06,1.084160e-06,1.362575e-06,1.298073e-06,5.357613e-07,3.364116e-07,3.311025e-07,1.439164e-07,4.063457e-08,1.761115e-08,8.868716e-09,2.765349e-09,5.057543e-10,5.419542e-11,4.823885e-12,0.767402,0.763228,0.72857,0.724856,0.720314,0.717113,0.749522,0.782215,0.7834,0.762521,0.756099,0.753244,-704.303772,93.277084,9.311788,12.922705,7.454835,17.394232,-4.342582,3.875228,-8.110920,-0.968131,-9.430532,-1.396702,2.581494,-1.416424,0.146414,-0.556509,-3.518421,0.420847,2.156458,-3.789995,-2.339234,-1.558570,1.003638,-2.425308,-2.732662,-2.345526,-1.238754,-2.41

In [27]:
X1 = X.copy()

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [29]:
MLPClassifierModeling(X_train, X_test, y_train, y_test)

Accuracy of the Recognizer is: 51.3%
              precision    recall  f1-score   support

     neutral       0.00      0.00      0.00         0
        calm       0.68      0.73      0.70        37
       happy       0.36      0.60      0.45        43
         sad       0.55      0.41      0.47        44
       angry       0.64      0.66      0.65        35
        fear       0.52      0.52      0.52        29
     disgust       0.45      0.57      0.51        42
    surprise       0.83      0.13      0.22        39

   micro avg       0.51      0.51      0.51       269
   macro avg       0.50      0.45      0.44       269
weighted avg       0.57      0.51      0.50       269



,actual_emotion,predicted_emotion
0,angry,angry
1,calm,disgust
2,happy,happy
3,sad,sad
4,fear,fear
5,disgust,disgust
6,sad,sad
7,calm,calm
8,sad,calm
9,fear,fear


**Improving with standardization**

In [30]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y, test_size=0.2, random_state=42)

In [31]:
#standardization

transformer = StandardScaler()
transformer.fit(X1_train)

# saving in a pickle
with open('std_transformer3.pickle', 'wb') as file:
    pickle.dump(transformer, file)
    
# loading from a pickle  
with open('std_transformer3.pickle', 'rb') as file:
    loaded_standardized = pickle.load(file)

X1_train = loaded_standardized.transform(X1_train)
X1_test = loaded_standardized.transform(X1_test)

In [32]:
MLPClassifierModeling(X1_train, X1_test, y1_train, y1_test)

Accuracy of the Recognizer is: 73.2%
              precision    recall  f1-score   support

     neutral       0.00      0.00      0.00         0
        calm       0.83      0.95      0.89        37
       happy       0.62      0.53      0.57        43
         sad       0.83      0.68      0.75        44
       angry       0.79      0.86      0.82        35
        fear       0.60      0.83      0.70        29
     disgust       0.71      0.64      0.67        42
    surprise       0.74      0.72      0.73        39

   micro avg       0.73      0.73      0.73       269
   macro avg       0.64      0.65      0.64       269
weighted avg       0.74      0.73      0.73       269



,actual_emotion,predicted_emotion
0,angry,angry
1,calm,calm
2,happy,surprise
3,sad,sad
4,fear,fear
5,disgust,disgust
6,sad,sad
7,calm,calm
8,sad,sad
9,fear,fear


The scores of our model have not improved without the neutral emotions but the prediction of the emotions seem to be more balanced.